NOTE: This is a Jupyter Notebook, so it won't be used in production, only for testing

Resources:
- https://quantum-computing.ibm.com/composer/files/new
- https://wybiral.github.io/quantum/
- https://jarrodmcclean.com/basic-quantum-circuit-simulation-in-python/
- https://quantumcomputing.stackexchange.com/questions/29454/why-do-quantum-computing-simulators-have-the-measurement-function

# Quantum Computation

In [96]:
import numpy as np
import scipy as sp

In [97]:
# operator matrices

# identity
Id = np.array([
    [1,0],
    [0,1]
])
# pauli x
X = np.array([
    [0,1],
    [1,0]
])
# pauli y
Y = np.array([
    [0,0-1j],
    [0+1j,0]
])
# pauli z
Z = np.array([
    [1,0],
    [0,-1]
])
# hadamard
H = 1/np.sqrt(2) * np.array([
    [1,1],
    [1,-1]
])
# phase
S = np.array([
    [1,0],
    [0,0+1j]
])
# pi over eight
T = np.array([
    [1,0],
    [0,np.exp((0+1j)*np.pi/4)]
])
# |0><0>
P_0 = np.array([
    [1,0],
    [0,0]
])
# |1><1>
P_1 = np.array([
    [0,0],
    [0,1]
])

In [98]:
class QState:
    def __init__(self, qbit_cnt, info=False):
        self.qbit_cnt = qbit_cnt
        # create state |00...0> with qbit_cnt amount of zeros
        self.state = np.zeros(2**self.qbit_cnt, dtype=complex)
        self.state[0] = 1

        if info:
            max_bin_width = len(bin(len(self.state) - 1)[2:])
            binary_str = bin(0)[2:].zfill(max_bin_width)
            print("Initial state: |{}>".format(binary_str))
    
    # show all states including the impossible one (unless
    # otherwise noted) and show according probabilities
    # DISCLAIMER: rounding/floating point errors may occur
    def show_state_and_probs(self, reduced=False):
        max_len = 0
        for element in self.state:
            if reduced and element == 0+0j:
                continue

            curr_len = max(len(str(element.real)), len(str(element.imag)))
            if curr_len > max_len:
                max_len = curr_len
        
        max_bin_width = len(bin(len(self.state) - 1)[2:])

        for i, element in enumerate(self.state):
            if reduced and element == 0+0j:
                continue

            real_part_str = "{:.{width}f}".format(element.real, width=max_len)
            imaginary_part_str = "{:.{width}f}".format(element.imag, width=max_len)
            
            formatted_element = ("{}+{}i".format(real_part_str, imaginary_part_str) if element.imag >= 0 else
                                 "{}{}i".format(real_part_str, imaginary_part_str))

            binary_str = bin(i)[2:].zfill(max_bin_width)

            prob_str = str(round((np.absolute(element)**2)*100, 4))

            if i > 0 and element.real >= 0:
                print("+{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            elif i > 0 and element.real < 0:
                print("{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            elif i <= 0 and element.real >= 0:    
                print(" {}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            else:
                print("{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))

    ###
    ### GATE CREATION HELPER ###
    ###
    
    # e.g. turns [A, B, C, D] into A ⊗ B ⊗ C ⊗ D
    def op_mats_arr_to_tens(self, op_mats_arr):
        result = np.array([[1.0]])
        for op_mat in op_mats_arr:
            result = np.kron(result, op_mat)
        
        return result

    def sing_qbit_op(self, pos, op_mat):
        op_mats_arr = (
              (pos)*[Id]
            + (1)*[op_mat]
            + (self.qbit_cnt-pos-1)*[Id]
        )
        gate = self.op_mats_arr_to_tens(op_mats_arr)
        self.state = np.matmul(gate, self.state)
    
    def qbit_0_or_1(self, pos):
        op_mats_arr = (
              (pos)*[Id]
            + (1)*[P_0]
            + (self.qbit_cnt-pos-1)*[Id]
        )
        gate = self.op_mats_arr_to_tens(op_mats_arr)

        if np.array_equal(np.matmul(gate, self.state), self.state):
            return 0
        return 1

    ###
    ### SINGLE-QUBIT GATES ###
    ###

    def pauli_x(self, pos):
        self.sing_qbit_op(pos, X)

    def pauli_y(self, pos):
        self.sing_qbit_op(pos, Y)

    def pauli_z(self, pos):
        self.sing_qbit_op(pos, Z)

    def hadamard(self, pos):
        self.sing_qbit_op(pos, H)
    
    def phase(self, pos):
        self.sing_qbit_op(pos, S)

    def pi_ov_8(self, pos):
        self.sing_qbit_op(pos, T)

    ###
    ### MULTI-QUBIT GATES ###
    ###

    # doesn't matter whether ctrl_pos smaller or bigger than targ_pos
    def cnot(self, ctrl_pos, targ_pos):
        if self.qbit_0_or_1(ctrl_pos) == 1:
            self.sing_qbit_op(targ_pos, X)

    def cz(self, ctrl_pos, targ_pos):
        if self.qbit_0_or_1(ctrl_pos) == 1:
            self.sing_qbit_op(targ_pos, Z)
    
    def cp(self, ctrl_pos, targ_pos):
        if self.qbit_0_or_1(ctrl_pos) == 1:
            self.sing_qbit_op(targ_pos, S)
    
    def toffoli(self, ctrl_pos1, ctrl_pos2, targ_pos):
        if self.qbit_0_or_1(ctrl_pos1) == 1 and self.qbit_0_or_1(ctrl_pos2) == 1:
            self.sing_qbit_op(targ_pos, X)

    def swap(self, pos1, pos2):
        # using special decomposition to achieve it's functionality:
        self.cnot(pos1, pos2)
        self.cnot(pos2, pos1)
        self.cnot(pos1, pos2)

    def fredkin(self, ctrl_pos, swap_pos1, swap_pos2):
        if self.qbit_0_or_1(ctrl_pos) == 1:
            self.swap(swap_pos1, swap_pos2)



In [99]:
### Testing

qs = QState(2, True)

qs.pauli_x(0)
qs.show_state_and_probs(True)
print(qs.qbit_0_or_1(0))
qs.show_state_and_probs(True)

# qs.pauli_x(1)
qs.show_state_and_probs(True)
print(qs.qbit_0_or_1(1))
qs.show_state_and_probs(True)


qs.show_state_and_probs(True)

Initial state: |00>
+1.000+0.000i|10>	-> 100.0%
1
+1.000+0.000i|10>	-> 100.0%
+1.000+0.000i|10>	-> 100.0%
0
+1.000+0.000i|10>	-> 100.0%
+1.000+0.000i|10>	-> 100.0%


# Protocol

In [100]:
input_info = {"qbit_cnt": 3}
input_gates = [{"type": "pauli_x", "qbit": [0]}, {"type": "cnot", "qbit": [0, 1]}]
# careful that in [0, 1] the first number MUST be the control qubit, and the other one the target qubit
# SO EVERYTHING MUST BE STRUCTURED ACCORDING TO THE GATE FUNCTIONS (that are listed in QState) 

qs = QState(input_info["qbit_cnt"], True)

for input_gate in input_gates:
    gate_func = getattr(qs, input_gate["type"])
    gate_func(*input_gate["qbit"])

qs.show_state_and_probs(True)

Initial state: |000>
+1.000+0.000i|110>	-> 100.0%
